<a href="https://colab.research.google.com/github/Quentin1168/StyleGan/blob/main/coord_to_img.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code used to translate a previously generated set of coordinates into iamges. I used Frederik Ueberschär's guide on how to sample a subset of satellite images and create a dataset.

https://ueberf.medium.com/a-closer-look-at-the-code-for-creating-a-dataset-of-satellite-images-for-stylegan-training-9c0e5088a303

In [ ]:
import ee
import urllib.request

#authenticate with earth engine
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=XReIJiR7bx5-yHuZaQl6qIaukVKDfRi5yVm455YMIvM&tc=3_Z7w_Fawmzt6vt9CI3RBS5HV0y_g8zCN9GggUed-1I&cc=GaXqFjSAT4_swtgI4VJnQLGtOMyQ6UW1OOhopS-A2J4

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWgavdch1yNb01aiPnb1IFUEqZjDrF0kC8g0mkCmIYO1VYi9MFpPdKNdibQ

Successfully saved authorization token.


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Exporting coast and land coordinates 

In [ ]:
!ls "/content/drive/My Drive"

'Colab Notebooks'


In [13]:
import pandas as pd

land_cords = pd.read_csv('/content/drive/My Drive/Colab Notebooks/map1.csv')
coast_cords = pd.read_csv('/content/drive/My Drive/Colab Notebooks/map2.csv')

frames = [land_cords, coast_cords]
result = pd.concat(frames, ignore_index=True)

#change start location based on how many images uploaded
start_location = 3673
print(result)

               Y           X     id  Unnamed: 3
0      80.213542  -63.295495      0         NaN
1      61.598523  -48.422729      1         NaN
2      20.283963   31.160884      2         NaN
3      61.634820 -142.341038      3         NaN
4     -86.130654 -152.938383      4         NaN
...          ...         ...    ...         ...
11005  77.799088  -71.943684  11005         NaN
11006  81.859523  -61.849311  11006         NaN
11007  73.330558 -106.651529  11007         NaN
11008  73.442652 -104.578254  11008         NaN
11009  72.790739 -105.339014  11009         NaN

[11010 rows x 4 columns]


In [15]:
vis_min = 0  #Visualization settings for the thumbnail
vis_max = 1700 #Visualization settings for the thumbnail
vis_bands = ['B4', 'B3', 'B2'] #Includes the bands for RGB 
imageDimensions = 512* 512 #Set thumbnail image size (can't be too big, or you run into problems)

for index, row in result.iloc[start_location:].iterrows(): 
  print(index, row['id'], row['X'], row['Y']) 

  #Choose your location to request an image from
  longitude = row['X']
  latitude = row['Y']
  id = row['id']


  center = ee.Geometry.Point(longitude,latitude)

  # Import Sentinel dataset
  s2 = (ee.ImageCollection("COPERNICUS/S2_SR")
    .filterBounds(center)
    .sort('CLOUDY_PIXEL_PERCENTAGE')
    .filterDate('2020-01-01', '2020-12-30')
    .first()
  )
    
    

  # Working, but with getInfo(), which makes the code a bit slower
  try:
    global sentinel_footprint
    sentinel_footprint = (ee.Geometry.Polygon((s2.getInfo().get('properties').get('system:footprint').get('coordinates'))))
    #print(sentinel_footprint)
  except:
    print("No Footprint found")
    continue


  # Create a rectangular export area
  exportAreaLarge = (sentinel_footprint.centroid().buffer(40000).bounds());
  exportAreaMedium = (sentinel_footprint.centroid().buffer(25000).bounds());
  exportAreaSmall = (sentinel_footprint.centroid().buffer(10000).bounds());
  exportAreaNofit = (sentinel_footprint.centroid().buffer(1000).bounds());

  # This part is nicer in the JavaScript version, but I did not get it running for Python yet

  if sentinel_footprint.contains(exportAreaLarge,1).getInfo() == True:
    print("Large fits")
    exportArea = exportAreaLarge
  elif sentinel_footprint.contains(exportAreaMedium,1).getInfo() == True:
    print("Medium fits")
    exportArea = exportAreaMedium
  elif sentinel_footprint.contains(exportAreaSmall,1).getInfo() == True:
    print("Small fits")
    exportArea = exportAreaSmall
  else:
    print("No fit")
    exportArea = exportAreaNofit
  #no fit images are pruned since they sometiems have artefacts or poor quality
  if exportArea != exportAreaNofit:
    s2Vis = {
        'region': exportArea,
        'crs': (s2.select('B4').projection()),
        'dimensions': imageDimensions,
        'format': 'jpg',
        'min': vis_min,
        'max': vis_max,
        'bands': vis_bands
    }

    s2_url = (s2.getThumbURL(s2Vis))

    #Change the location where the images are saved by replacing "content" with the location in your Google Drive
    s2_name = "/content/drive/My Drive/Colab Notebooks/sentinel/S2_{}_{}*{}.jpg".format(id,longitude,latitude)


    urllib.request.urlretrieve(s2_url, s2_name)
    print(s2_url)

Streaming output truncated to the last 5000 lines.
4988 4988.0 159.05459154742 -83.2179668622227
No Footprint found
4989 4989.0 60.0152272124179 -72.6821622600698
Large fits
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/7ce6be21e51b8c714db0c933dda509b9-1c284a409de52d29e81bb28384e72e70:getPixels
4990 4990.0 90.9509656546265 -77.8865041723996
No Footprint found
4991 4991.0 23.1719742182158 27.1673216449196
Large fits
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/6497fcadd9326987998677f0decdd883-e75368a10c2975b403523fa7344631e0:getPixels
4992 4992.0 9.25998974995358 56.1906630685085
Large fits
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/761b788028f08c7efa730935171792d9-9c5a8f06569c7626903768d8f3220c32:getPixels
4993 4993.0 106.433220078769 -73.3968793783125
Large fits
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/2cdba2d0b04884bc48f84bb11616c688

Streaming output truncated to the last 5000 lines.
9303 9303.0 -99.91522834 16.79561815
Large fits
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/3cdddcb2fdef3ee6687dee670b3a0c42-dd35efe173420d126a9753add34a87f3:getPixels
9304 9304.0 -83.24396983 10.23061204
Large fits
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/fbf791844ea3b63cbddb9d7f44ca0bb8-6c44d44839d4636a79719efe59479b9c:getPixels
9305 9305.0 -123.8077151 39.42745047
Large fits
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/4c1d379755b2b23b047eaf043c4faf91-fd4c5de1e14cf866659ed8a924591f0e:getPixels
9306 9306.0 -115.1340039 68.86349409
Large fits
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/210da2a7c71c5cbff38970761e3aff19-e7004ed9f6af9d53a91101d7c5a56d19:getPixels
9307 9307.0 -161.9992752 58.88173705
Large fits
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbn

Code to prune images that are completely white (suspected to be artic photos), there were too many images that were just completely white, which could ruin the generation of samples (technically still satellite images, but it would be very monotone).




In [54]:
from PIL import Image
import numpy as np
import os
imageDimensions = 512*512
path = "/content/drive/My Drive/Colab Notebooks/sentinel/"
for filename in os.listdir(path):
  img = np.asarray(Image.open(path+filename))
  # the number 750000 is calculated from seeing different samples of nearly white images and their corresponding pixel value
  # i.e. white images with 1 to 2 tiny lakes were disregarded, 
  if np.sum(img == 255) > 750000:
    print(filename + " is a white image")
    os.remove(path+filename)


S2_179.0_143.684363490501*67.7052048161145.jpg is a white image
S2_240.0_79.2292476242955*50.0779792965794.jpg is a white image
S2_242.0_24.7150699334933*-70.4200023973938.jpg is a white image
S2_245.0_80.1056966209553*-70.5587851757894.jpg is a white image
S2_263.0_68.044508031978*-75.5705294117002.jpg is a white image
S2_266.0_18.1332317568965*-72.9175947645763.jpg is a white image
S2_268.0_48.1860696287778*-69.3693546272475.jpg is a white image
S2_271.0_138.372402481302*-79.7852542931493.jpg is a white image
S2_277.0_123.861200917116*-76.1784527638318.jpg is a white image
S2_283.0_-42.4147478461695*81.5332329374668.jpg is a white image
S2_284.0_-142.346219805337*-76.0682297474799.jpg is a white image
S2_293.0_-119.661037808183*68.1444191965199.jpg is a white image
S2_300.0_124.747597857731*-71.1738203968549.jpg is a white image
S2_310.0_10.1298262124307*-76.9480202722305.jpg is a white image
S2_312.0_98.5132544781173*-72.3268607459695.jpg is a white image
S2_316.0_-1.87186911821283*